# Phase Vocoder - Time Scale & Pitch Shift with Short Time Fourier Transforms

This is a demonstration of stretching, shrinking, and resampling audio with a phase vocoder.
The algorithm uses the formulas and descriptions from:


 J. Laroche and M. Dolson (1999). "Improved Phase Vocoder Time-Scale Modification of Audio". IEEE Transactions on Speech and Audio Processing. 7 (3): 323–332. doi:10.1109/89.759041.

In [ ]:
%matplotlib inline

import numpy as np
import numpy.random as nprandom
import scipy.io.wavfile
import scipy.signal
import scipy.fft
import scipy.signal.windows
import matplotlib.pyplot as plt
import IPython
import pdb
from ipywidgets import widgets

plt.rcParams['figure.figsize'] = [24, 12]
plt.style.use('ggplot')
np.set_printoptions(precision=3, suppress=True)

In [ ]:
# load up a wav file containing our audio
y2k_sample_rate, year_2000_data = scipy.io.wavfile.read("./audio/year2000.wav")
IPython.display.Audio(data = year_2000_data, rate = y2k_sample_rate)

# Short Time Fourier Transform

## Analysis Step
$$X(t_a^u, \Omega_k) = \sum_{n=-\infty}^{\infty}h(n)x(t_a^u + n)e^{-j\Omega_k n}$$
Where:

$x$ is the input signal.

$h(n)$ is the analysis Hann window.

$\Omega_k = \frac{2 \pi k}{N}$ is the frequency of the kth channel.

$N$ is the DFT size.



## Synthesis Step
$$y(n) = \sum_{u=-\infty}^{\infty} y_u(n-t_s^u)$$

$$y_u(n) = \frac{1}{N} \sum_{k=0}^{N-1} Y(t_s^u, \Omega_k) e^{j\Omega_k n}$$

This stft is modified from the analysis stft by:


$$|Y(t_s^u, \Omega_k)| = |X(t_a^u, \Omega_k)|$$

$$\angle Y(t_s^u, \Omega_k) = \angle Y(t_s^{u-1}, \Omega_k) + R_s \hat{\omega}_k(t_a^u)$$

$$\hat{\omega}_k(t_a^u) = \Omega_k + \frac{1}{R_a}\Delta_p \Phi_k^u$$

$$ \Delta \Phi_k^u = \angle X(t_a^u, \Omega_k) - \angle X(t_a^{u-1}, \Omega_k) - R_a \Omega_k $$

$\Delta_p \Phi_k^u$ is $\Delta \Phi_k^u$ unrawpped between $\pm \pi$.

$t_a^u = uR_a$ where $R_a$ is the hop factor during analysis. 

$t_s^u = uR_s$ where $R_s$ is the hop factor during the synthesis stage.

$t_s^u = \alpha t_a^u$ for some scaling factor $\alpha$. So, $R_s = \alpha R_a$.


In [ ]:
N = 2**12
R_a = N//4

In [ ]:
def short_time_fourier_transform(data, window_size, analysis_hop_size):
        
    total_windows = (len(data) - window_size) // analysis_hop_size
    # create output buffer, a 2D array where each row is the FFT of the next slice of data
    output = np.zeros((total_windows, window_size), dtype=complex)
    # create the window to smooth  the FFT segments
    window = scipy.signal.windows.hann(window_size, sym=False)

    # break data into overlapping windowed chunks, perform fft on each chunk, store results as a rows
    for hop in range(0, total_windows):
        hop_location = hop * analysis_hop_size
        windowed_data = window * data[hop_location: hop_location + window_size]
        fft = scipy.fft.fft(windowed_data)
        output[hop] = fft
        
    return output

In [ ]:
y2k_stft = short_time_fourier_transform(year_2000_data, N, R_a)

In [ ]:
# plot the STFT over Time
plt.pcolormesh(np.abs(y2k_stft.T), cmap=plt.get_cmap('magma'))
plt.ylim(0, 500)

## Phase Vocoder - Inverse STFT with Different Hop Size

In [ ]:
def synthesis_phase_spectrum(stft, analysis_hop_size, synthesis_hop_size, omegas):
    TAU = 2 * np.pi
    phases = np.angle(stft)
    delta_phases = phases - np.roll(phases, 1, axis=0) - (analysis_hop_size * omegas)
    unwrapped_delta_phases = delta_phases - TAU * np.round(delta_phases / TAU)
    instantaneous_frequencies = omegas + (unwrapped_delta_phases / analysis_hop_size)
    for i in range(1, len(stft)):
        phases[i] = phases[i-1] + (synthesis_hop_size * instantaneous_frequencies[i])
        
    return phases

In [ ]:

def phase_vocoder_istft(stft, analysis_hop_size, factor):
    
    synthesis_hop_size = int(analysis_hop_size * factor)
    
    num_frames = len(stft)
    N = len(stft[0])  # DFT size
    
    window = scipy.signal.windows.hann(N, sym=False)
    omegas = (2 * np.pi * np.arange(N)) / N
    
    magnitude_spectrum = np.abs(stft)
    synth_phase_spectrum = synthesis_phase_spectrum(stft, analysis_hop_size, synthesis_hop_size, omegas)
    
    output = np.zeros(synthesis_hop_size * (num_frames - 1) + N)
    
    # change the current frame to use the same magnitudes from the analysis step but the adjusted phase values
    corrected_spectrums = magnitude_spectrum * np.exp(synth_phase_spectrum * 1j)
    
    # Now, do inverse FFTs at intervals of the synthesis hop size, summing the results together.
    for hop in range(0, num_frames):
        ifft = np.real(scipy.fft.ifft(corrected_spectrums[hop]))
        hop_location = hop * synthesis_hop_size
        output[hop_location: hop_location + N] += window * ifft

    return output


In [ ]:
voice_stretch_factor = 1.122462
stretched = phase_vocoder_istft(y2k_stft, R_a, voice_stretch_factor)

In [ ]:
IPython.display.Audio(data = stretched, rate = y2k_sample_rate)

Now, autotune the voice up by playing the stretched version at a faster rate.

In [ ]:
IPython.display.Audio(data = stretched, rate = y2k_sample_rate * voice_stretch_factor)

# Pitch Changing Examples

Musical pitches are related to each other by adjacent semitones having a ratio of $2^{1/12}$. So, an $n$ semitone shift is equal to $(2^{1/12})^n = 2^{n/12}$.

In [ ]:
# helper function to convert a number of semitones to pitch shift into the 
# length factor that the phase vocoder algorithm expects
def semitones_to_ratio(n):
    return np.power(2, n/12)

In [ ]:
# function that makes time stretched and pitch scaled version of
# the input file name and then displays IPython Audio elements for them
def show_autotune_example(path, dft_size, analysis_hop, factor):
    fs, data = scipy.io.wavfile.read(path)
    stft = short_time_fourier_transform(data, N, analysis_hop)
    stretched = phase_vocoder_istft(stft, analysis_hop, factor)
    out = [
    (IPython.display.Audio(data = data, rate = fs)),
    (IPython.display.Audio(data = stretched, rate = fs)),
    (IPython.display.Audio(data = stretched, rate = fs * factor))]
    for x in out:
        IPython.display.display(x)

## Bach - Violin Concerto in A minor

First, we speed up this performance of Bach's Violin Concerto in A Minor, but keep it _in_ A minor. :)
Then, if we speed the audio but scale the sample rate by the same amount we scaled the duration, we get a pitch shift (Auto-Tune)!

In [ ]:
show_autotune_example("./audio/BachConcertoAMinor_Intro.wav", N, R_a, semitones_to_ratio(-3))

## Intervals - I'm Awake

In [ ]:
show_autotune_example("./audio/IntervalsImAwake.wav", N, R_a, semitones_to_ratio(-2))

## The Dead South - In Hell I'll be in Good Company

In [ ]:
show_autotune_example("./audio/HellGoodCompany.wav", N, R_a, semitones_to_ratio(-2))

## Beethoven - Moonlight Sonata, 3rd movement

In [ ]:
show_autotune_example("./audio/moonlight.wav", N, R_a, semitones_to_ratio(2))

## Nikolai Rimsky-Korsakov - Flight of the Bumblebee

In [ ]:
show_autotune_example("./audio/bumblebee.wav", N, R_a, 165/325)

## Christopher Tin - Temen Oblak("Dark Clouds")

In [ ]:
show_autotune_example("./audio/DarkClouds.wav", N, R_a, semitones_to_ratio(-5))

In [ ]:
fs, data = scipy.io.wavfile.read("./audio/bumblebee.wav")
stft = short_time_fourier_transform(data, N, R_a)
stretched = phase_vocoder_istft(stft, R_a, 165/325)
stretched/=np.max(stretched)
#IPython.display.Audio(data=stretched, rate=fs)
#scipy.io.wavfile.write("fastBee.wav", fs, stretched)
    